In [ ]:
import os

from matplotlib import ticker
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d.art3d import pathpatch_2d_to_3d, Poly3DCollection
import numpy as np
try:
    import open3d as o3d
except ImportError:
    import sys
    print(sys.exc_info())
import pandas as pd

from dosipy.utils.dataloader import load_ear_data
from dosipy.utils.viz import (set_colorblind, fig_config, set_axes_equal,
                              save_fig)
from helpers import (clean_df, export_pcd, export_fields, poynting_vector,
                     export_rect_idx, get_imcolors)

In [ ]:
set_colorblind()
%config InlineBackend.figure_format = 'retina'

# SAR distribution

In [ ]:
# input data

polarization = 'te'
f = 26

In [ ]:
# load model data

fname_block = os.path.join('data', f'loss_3D_ear_{f}GHz_{polarization.upper()}_front.txt')
df = (pd.read_csv(fname_block, names=['x [mm]', 'y [mm]', 'z [mm]', 'Pl [W/m^3]'],
                  header=None, delim_whitespace=True, skiprows=[0, 1])
      [lambda x: (x['Pl [W/m^3]'] != 0.0)])
df.reset_index(drop=True, inplace=True)
xyz = export_pcd(df)
Pl = df['Pl [W/m^3]'].to_numpy()
tissue_density = 1049

# postprocessing
SAR = Pl / tissue_density

In [ ]:
skip = 2
scaler = 2
fig_config(latex=True, text_size=20, scaler=scaler)
fig = plt.figure()
ax = plt.axes(projection ='3d')
cs = ax.scatter(xyz[::skip, 2], xyz[::skip, 0], xyz[::skip, 1],
                c=SAR[::skip],
                cmap='viridis',
                s=0.5,
               )
cbar = fig.colorbar(cs, shrink=0.55, pad=0)
cbar.ax.set_ylabel('$SAR$ [W/kg]')
cbar.ax.yaxis.labelpad = 12
ax.set(xlabel='$z$ [mm]', ylabel='$x$ [mm]',
       xticks=[10, -10, -30],
       yticks=[50, 70, 90],
       zticks=[0, 20, 40])
ax.zaxis.set_rotate_label(False)
ax.set_zlabel('$y$ [mm]', rotation=90)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 15
ax.yaxis.labelpad = 15
ax.zaxis.labelpad = 12
ax.view_init(10, 70)
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
fig.tight_layout();

# save figure
#fname = os.path.join('figures', f'{polarization}{f}_sar')
#save_fig(fig, fname=fname, formats=['png'])